In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import json
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from io import BytesIO
import base64

[nltk_data] Downloading package punkt to /Users/dhv001/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', -1)

In [3]:
# data csv from Kaggle notebook instead of pickling
df = pd.read_csv('data.csv')

In [4]:
len(df)

1301

In [5]:
df.head()

Unnamed: 0                                  paper_id  \
0  0           b891efc6e1419713b05ff7d89b26d260478c28df   
1  1           7a5d731cd59597da7bcef171af936c0af4d7528c   
2  2           854545208a6bc7cafa8744604899e32237ff2047   
3  3           e8d81153777e9f92cbc2bc9ea03ed1f0d0b78543   
4  4           a14b5655cb13ed64cb8cff7c806a7b58c858b8b7   

                                                                                                                                                                                           title  \
0  6756    tuberculosis prevention in healthcare workers in china  years after the severe acute respiratory syndrome pandemic\nName: title, dtype: object                                          
1  epidemic and case investigations surveillance operation for the 141st confirmed case of middle east respiratory syndrome coronavirus in response to the patient's prior travel to jeju island   
2  highly pathogenic avian influenza a virus h5n1 non-structural protein 1 is associated with apoptotic activation of the intrinsic mitochondrial pathway                                          
3  group b betacoronavirus in rhinolophid bats, japan                                                                                                                                              
4  feasibility of controlling covid-19 outbreaks by isolation of cases and contacts                                                                                                                

                                                                                                                                                                                                                                                                                                              author_list  \
0  []                                                                                                                                                                                                                                                                                                                       
1  ['Open Access', 'Jong-Myon Bae']                                                                                                                                                                                                                                                                                         
2  ['Qian Bian', 'Jing Lu', 'L I Zhang', 'Ying Chi', 'Yan Li', 'Hongxiong Guo']                                                                                                                                                                                                                                             
3  ['Jin Suzuki', 'Ryota Sato', 'Tomoya Kobayashi', 'Toshiki Aoi', 'Ryô Harasawa']                                                                                                                                                                                                                                          
4  ['J Hellewell', 'S Abbott', 'Jarvis Phd', 'T W Russell', 'J D Munday', 'A J Kucharski', 'W J Edmunds', 'S Funk', 'Rosalind M Eggo', 'Joel Hellewell', 'Sam Abbott', 'Amy Gimma', 'Nikos I Bosse', 'Christopher I Jarvis', 'Timothy W Russell', 'James D Munday', 'Adam J Kucharski', 'John Edmunds', 'Sebastian Funk']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [6]:
class Preprocesser:
    def __init__(self, df):
        self.keywords = ['incident command system',
                         'emergency operations',
                         'joint information center',
                         'social distancing',
                         'childcare closers',
                         'travel advisory',
                         'travel warning',
                         'isolation',
                         'quarantine',
                         'mass gathering cancellations',
                         'school closures',
                         'facility closures',
                         'evacuation',
                         'relocation',
                         'restricting travel',
                         'travel ban',
                         'patient cohort',
                         'npi']
        self.occurances_minimum = 2
        self.df_full = df
        print(self.df_full.shape)
        self.key_slice()
        print(self.df_full.shape)
        self.npi_slice()
        print(self.df_full.shape)
    
    def key_slice(self):
        self.df_full = self.df_full[self.df_full['abstract'].str.contains('|'.join(self.keywords), na=False, regex=True)].reset_index(drop=True)
        
    def npi_slice(self):
        def get_count(row):
            return sum([row['abstract'].count(keyword) for keyword in self.keywords])
        self.df_full = self.df_full[self.df_full.apply(get_count, axis=1) >= self.occurances_minimum]
        
    def remove_stopwords(self,columns):
        stop = stopwords.words('english')
        for col in columns:
            self.df_full[col] = self.df_full[col].astype(str).apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    def to_tfidf(self,columns):
        for col in columns:
            tfidfv = TfidfVectorizer()
            self.df_full[col + '_tfidf'] = list(tfidfv.fit_transform(self.df_full[col]).toarray())
            
    def remove_punc(self, columns):
        for col in columns:
            self.df_full[col] = self.df_full[col].str.replace('[^a-zA-Z\s]+','')
        
def display_wordcloud(text):
    wc= WordCloud(max_font_size=50,
                          max_words=100,
                          background_color='white',
                          width=480, height=360).generate(text)
    return wc.to_image()

In [7]:
def pca_apply(df, columns, n_comp):
    new_df = df.copy()
    for col in columns:
        pca = PCA(n_components=n_comp, random_state=1)
        new_df[col+'_pca'] = list(pca.fit_transform(np.stack(df[col].to_numpy())))
    return new_df.reset_index(drop=True)

def apply_scaler(df, columns):
    new_df = df.copy()
    for col in columns:
        scaler = StandardScaler()
        new_df[col + '_scaled'] = list(scaler.fit_transform(np.stack(df[col].to_numpy())))
    return new_df.reset_index(drop=True)

def tokenize(row):
    title_tokens = []
    title = row['abstract']
    if title == title:
        title = re.sub('(/|\|:|&|#|-|\.)', '', title)
        tokens = word_tokenize(title)
        remove_sw = [word for word in tokens if word not in stopwords.words('english')]
        remove_numbers = [word for word in remove_sw if not word.isnumeric()]
        remove_comas = [word for word in remove_numbers if not word in [',', '(', ')', '"', ':', '``', '.', '?']]
        title_tokens.extend(remove_comas)
    return [value[0] for value in Counter(title_tokens).most_common()[0:30]]

In [8]:
prepr = Preprocesser(df)

(1301, 7)
(1301, 7)
(428, 7)


In [9]:
prepr.remove_punc(['body_text','abstract'])
prepr.remove_stopwords(['body_text', 'abstract'])
prepr.to_tfidf(['body_text', 'abstract'])
pca_df = pca_apply(prepr.df_full, ['abstract_tfidf','body_text_tfidf'], 3)
scaled_df = apply_scaler(pca_df,['abstract_tfidf_pca','body_text_tfidf_pca'])
scaled_df['tokens'] = scaled_df.apply(tokenize, axis=1)

In [10]:
scaled_df.head()

Unnamed: 0                                  paper_id  \
0  1           7a5d731cd59597da7bcef171af936c0af4d7528c   
1  4           a14b5655cb13ed64cb8cff7c806a7b58c858b8b7   
2  11          69ca8c0a79935fbe985f02fc53c1a9da5c42acd3   
3  16          335dbf8f43a8eca80f3e2730b3f8a1e655ca2d14   
4  17          87b8b85391605bcd1b6c7ddaf1189b3229db4e06   

                                                                                                                                                                                           title  \
0  epidemic and case investigations surveillance operation for the 141st confirmed case of middle east respiratory syndrome coronavirus in response to the patient's prior travel to jeju island   
1  feasibility of controlling covid-19 outbreaks by isolation of cases and contacts                                                                                                                
2  effect of isolation practice on the transmission of middle east respiratory syndrome coronavirus among hemodialysis patients a 2-year prospective cohort study                                  
3  the characteristics of middle eastern respiratory syndrome coronavirus transmission dynamics in south korea                                                                                     
4  media impact switching surface during an infectious disease outbreak                                                                                                                            

                                                                                                                                                                                                                                                                                                              author_list  \
0  ['Open Access', 'Jong-Myon Bae']                                                                                                                                                                                                                                                                                         
1  ['J Hellewell', 'S Abbott', 'Jarvis Phd', 'T W Russell', 'J D Munday', 'A J Kucharski', 'W J Edmunds', 'S Funk', 'Rosalind M Eggo', 'Joel Hellewell', 'Sam Abbott', 'Amy Gimma', 'Nikos I Bosse', 'Christopher I Jarvis', 'Timothy W Russell', 'James D Munday', 'Adam J Kucharski', 'John Edmunds', 'Sebastian Funk']   
2  ['Hayne Cho Park', 'Sang-Ho Lee', 'Juhee Kim', 'Hyoung Kim', 'Ajin Cho', 'Hee Jung Jeon', 'Jieun Oh', 'Jung-Woo Noh', 'Da-Wun Jeong', 'Yang-Gyun Kim', 'Chang-Hee Lee', 'Kyung Don Yoo', 'Young-Ki Lee']                                                                                                                 
3  ['Yunhwan Kim', 'Sunmi Lee', 'Chaeshin Chu', 'Seoyun Choe', 'Saeme Hong', 'Youngseo Shin']                                                                                                                                                                                                                               
4  ['Yanni Xiao', 'Sanyi Tang', 'Jianhong Wu', 'S T ']                                                                                                                                                                                                                                                                      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
# from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

def get_breaks(row, col, word_limit=45, break_char='<br>', colon=True):
    if row[col] == row[col]:
        data = ''
        if col == 'tokens':
            words = row[col]
        elif colon:
            data = f'{col}: '
            words = row[col].split(' ')
        else:
            words = row[col].replace('<br>', '').split(' ')
        total_chars = 0

        # add break every length characters
        for i in range(len(words)):
            total_chars += len(words[i])
            if total_chars > word_limit:
                data += f'{break_char}{words[i]}'
                total_chars = 0
            else:
                if col == 'tokens' and data:
                    data += f', {words[i]}'
                else:
                    data += f' {words[i]}'
        return data.strip()
    return row[col]

class Cluster_Plot:
    def __init__(self, df, text_type, clust_nums):
        self.styles = {
            'pre': {
                'border': 'thin lightgrey solid',
                'overflowX': 'scroll'
            }
        }
        self.set_app()
        self.df = df
        self.set_text(text_type)
        self.clust_nums = clust_nums
        self.create_cluster_df()
        
        if self.app is not None and hasattr(self, 'callbacks'):
            self.callbacks(self.app)

    def run_process(self):
        self.set_app_layout()
        self.app.run_server()
        
    def set_app(self):
        self.app = dash.Dash(__name__,
                        external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css"])
        
    def set_app_layout(self):
        self.app.layout = html.Div(
            [
                html.Div([
                    html.H1('NPI Cluster Analysis'),
                    html.Label('Query by Abstract or Body'),
                    dcc.RadioItems(
                        id='abstract_or_body',
                        options=[{'label': 'abstract', 'value': 'abstract'},
                                 {'label': 'body', 'value': 'body_text'}],
                        value=self.text_type
                    ),
                    html.Label('Number of Clusters'),
                    dcc.Dropdown(
                        id='cluster_num',
                        options=[{'label': i+1, 'value': i+1} for i in list(range(20))],
                        value=self.clust_nums
                    ),
                    html.Label('Cluster Selection'),
                    dcc.Dropdown(
                        id='cluster_id',
                        options=self.cluster_id_list,
                        value='all'
                    )
                ], className='two columns'),
                
                html.Div([
                    dcc.Graph(id='graph', style={'width': "90%", 'display': 'inline-block'})
                ], className='five columns'),
                html.Div([
                    html.H3('Word Cloud for clicked data point'),
                    html.Img(id="image_wc")
                ], className='six columns'),
                
                html.Div([
                    html.Label('Search by keyword'),
                    dcc.Input(id='search',
                              value='',
                              type='text'),

                    html.Div(className='row', children=[
                        html.Div([
                            dcc.Markdown('''
                                **Hover Data**

                                Click on values in the plot.
                            '''),
                            html.Pre(id='hover-data', style=self.styles['pre'])
                        ])
                    ])
                ], className='row')
            ]
        )
        
    def create_cluster_df(self):
        self.cluster_id_list = [{'label': i, 'value': i} for i in list(range(self.clust_nums))]
        self.cluster_id_list.append({'label': 'all', 'value': 'all'})
        new_df = self.df.copy()
        kmeans = KMeans(n_clusters = self.clust_nums, random_state=1)
        new_df[self.col_cluster_id] = list(kmeans.fit_predict(np.stack(new_df[self.col].to_numpy())))
        self.cluster_df = new_df.reset_index(drop=True)
        self.cluster_df['title'] = self.cluster_df.apply(get_breaks, args=('title',), axis=1)
        self.cluster_df[['x', 'y', 'z']] = pd.DataFrame(self.cluster_df[self.col].values.tolist(),
                                                        index = self.cluster_df.index)
    
    def set_text(self, text_type):
        self.text_type = text_type
        self.col = f'{self.text_type}_tfidf_pca_scaled'
        self.col_cluster_id = f'{self.text_type}_tfidf_pca_scaled_clusterID'
        
    def callbacks(self, app):
        @app.callback([Output('cluster_id', 'options'),
                       Output('graph', 'figure')],
                      [Input('abstract_or_body', 'value'),
                       Input('cluster_num', 'value'),
                       Input('cluster_id', 'value'),
                       Input('search', 'value')])
        def update_clusters(abstract_or_body, cluster_num, cluster_id, search_string):
            self.set_text(abstract_or_body)
            self.clust_nums = cluster_num
            self.create_cluster_df()
            options = self.cluster_id_list
            df = self.cluster_df.copy()
            show_scale = True
            if cluster_id != 'all':
                show_scale = False
                df = df[df[self.col_cluster_id] == cluster_id]
            if search_string:
                mask = df['tokens'].apply(lambda x: search_string in x)
                df = df[mask]
            fig = px.scatter_3d(df, x='x', y='y', z='z',
                                 color=self.col_cluster_id,
                                 hover_name='title',
                                 hover_data=['paper_id', 'doi'])
            fig.update_layout(scene = dict(
                                xaxis = dict(nticks=4, range=[-5,5],),
                                yaxis = dict(nticks=4, range=[-5,5],),
                                zaxis = dict(nticks=4, range=[-5,5],),),
                              hoverlabel=dict(
                                bgcolor='white', 
                                font_size=8, 
                                font_family='Rockwell'
                              ),
                              coloraxis=dict(
                                colorbar=dict(title='Cluster ID'),
                                showscale=show_scale
                              ))
            return options, fig
        
        @app.callback([Output("hover-data", "children"),
                      Output('image_wc', 'src')],
                      [Input("graph", "clickData"),
                       Input('image_wc', 'id')])
        def display_click_data(clickData, b):
            string = None
            if clickData:
                click_index = clickData['points'][0]['pointNumber']
                token_string = ''
                if self.cluster_df.iloc[click_index]["tokens"]:
                    token_string = get_breaks(self.cluster_df.iloc[click_index],
                                              'tokens',
                                              word_limit=100,
                                              break_char='\n')
                    token_string = f'Keywords: {token_string}'
                    
                string = get_breaks(self.cluster_df.iloc[click_index],
                                    'title',
                                    word_limit=100,
                                    break_char='\n',
                                    colon=False)
                item_list = ['abstract', 'body_text', 'author_list', 'paper_id', 'doi']
                for i in item_list:
                    formatted_data = get_breaks(self.cluster_df.iloc[click_index],
                                               i,
                                               word_limit=100,
                                               break_char='\n')
                    string += f'\n\n{formatted_data}'
                img = BytesIO()
                display_wordcloud(self.cluster_df.iloc[click_index][self.text_type]).save(img, format='PNG')
                image = 'data:image/png;base64,{}'.format(base64.b64encode(img.getvalue()).decode())
                if token_string:
                    string = f'{token_string}\n\n{string}'
                return string, image
            return string, None

c = Cluster_Plot(scaled_df, 'abstract', 10)
c.run_process()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Apr/2020 10:37:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 10:37:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 10:37:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 10:37:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 10:37:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 10:37:43] "POST /_dash-update-component HTTP/1.1" 200 -
